## 使用python实现kNN近邻算法

### 1.处理数据
首先我们需要读取数据。csv格式是没有任何标题，我们可以使用open函数来打开文件，使用csv模块对数据进行读写

In [6]:
import csv
with open('E:/Kaggle/iris.data', 'rb') as csvfile:
    lines = csv.reader(csvfile)

接下来我们需要将数据划分为训练集和测试集。首先我们先要将字符串转换成为数字，然后我们需要将数据集随机划分成为训练集和测试集，我们采用训练集和测试的比例是66/34。

之后我们将定义一个函数loadDataset，这个函数的作用是加载csv数据，按照66/34的比例随机划分训练集和测试集合

In [7]:
import csv
import random

def load_dataset(filename, split, trainset=[], testset=[]):
    with open(filename) as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        for x in range(len(dataset)-1):
            for y in range(4):
                dataset[x][y] = float(dataset[x][y])
            if random.random() < split:
                trainset.append(dataset[x])
            else:
                testset.append(dataset[x])

下载iris数据到本地路径，我们可以使用iris数据测试一下loadDataset函数：

In [9]:
trainset = []
testset = []
load_dataset('E:/Kaggle/iris.data', 0.66, trainset, testset)
print ('Train:' + repr(len(trainset)))
print ('Test:' + repr(len(testset)))

Train:105
Test:44


### 2.相似度
为了预测我们必须计算任意两个数据实例之间的相似性。这可以让我们在训练集中给定测试数据找到与k个最相似的数据实例。

鉴于所有测量花的数据都是具有相同的单位(所以这里不用进行归一化处理)，我们可以直接使用欧氏距离度量。这里定义欧氏距离为连个数组之间的平方差之和的平方根。

此外，我们想要控制哪些特征包含在距离计算。具体来说，我们只想包含前四个属性。一种方式是将欧氏距离限制为固定长度，忽略最终的维度。

In [10]:
import math

def euclidean_distance(instance1, instance2, length):
    distance = 0
    for x in range(length):
        distance += pow(instance1[x] - instance2[x], 2)
    return math.sqrt(distance)

我们可以使用一些样例数据来测试euclideanDistance函数

In [11]:
data1 = [2, 2, 2, 'a']
data2 = [4, 4, 4, 'b']
distance = euclidean_distance(data1, data2, 3)
print ('Distance:' + repr(distance))

Distance:3.4641016151377544


### 3.近邻
现在我们有相似度的度量，我们可以在给定一个未知的数据集，用来汇集k个最相似的实例。

这是一个直接的过程，计算所有实例的距离，并选择含有最小距离值的子集。

下面的getNeighbours函数是根据一个给定的测试实例，返回的k个最相似的近邻集。

In [12]:
import operator

def get_neighbours(trainset, test_instance, k):
    distances = []
    length = len(test_instance) - 1
    for x in range(len(trainset)):
        dist = euclidean_distance(test_instance, trainset[x], length)
        distances.append((trainset[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbours = []
    for x in range(k):
        neighbours.append(distances[x][0])
    return neighbours

我们可以测试getNeighbours函数，如下所示：

In [14]:
trainset = [[2, 2, 2, 'a'], [4, 4, 4, 'b']]
test_instance = [5, 5, 5]
k = 1
neighbours = get_neighbours(trainset, test_instance, 1)
print ((neighbours))

[[4, 4, 4, 'b']]


### 4.回应
一旦我们找到了和测试实例最相似的近邻集，下一步根据这些近邻集设计一个回应。

我们可以做到这一点，允许每个近邻投票他们的类的属性，并采取获得多数票作为预测结果。

In [16]:
import operator

def get_response(neighbours):
    classvotes = {}
    for x in range(len(neighbours)):
        response = neighbours[x][-1]
        if response in classvotes:
            classvotes[response] += 1
        else:
            classvotes[response] = 1
        sortedvotes = sorted(classvotes.items() ,key=operator.itemgetter(1), reverse=True)
        return sortedvotes[0][0]

我们可以使用测试数据来测试getResponse函数

In [17]:
neighbours = [[1, 1, 1, 'a'], [2, 2, 2, 'a'], [3, 3, 3, 'b']]
response = get_response(neighbours)
print ((response))

a


### 5.准确率
我们已经实现了kNN算法的各个部分，剩下的一个重要的问题就是如何评估预测的准确性。

一个简单的方法来评估该模型的准确性是计算正确的预测占所有预测的比率。

In [18]:
def get_accuracy(testset, predictions):
    correct = 0
    for x in range(len(testset)):
        if testset[x][-1] is predictions[x]:
            correct += 1
    return (correct/float(len(testset))) * 100

我们可以通过一个测试数据和预测数据来测试getAccuracy函数

In [19]:
testset = [[1, 1, 1, 'a'], [2, 2, 2, 'a'], [3, 3, 3, 'b']]
predictions = ['a', 'a', 'a']
accuracy = get_accuracy(testset, predictions)
print ((accuracy))

66.66666666666666


kNN算法完整的代码如下

In [22]:
import csv
import random
import math
import operator

def load_dataset(filename, split, trainset=[], testset=[]):
    with open(filename) as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        for x in range(len(dataset)-1):
            for y in range(4):
                dataset[x][y] = float(dataset[x][y])
            if random.random() < split:
                trainset.append(dataset[x])
            else:
                testset.append(dataset[x])

def euclidean_distance(instance1, instance2, length):
    distance = 0
    for x in range(length):
        distance += (instance1[x] - instance2[x]) ** 2
    return math.sqrt(distance)

def get_neighbours(trainset, test_instance, k):
    distances = []
    length = len(test_instance) - 1
    for x in range(len(trainset)):
        dist = euclidean_distance(test_instance, trainset[x], length)
        distances.append((trainset[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbours = []
    for x in range(k):
        neighbours.append(distances[x][0])
    return neighbours

def get_response(neighbours):
    classvotes = {}
    for x in range(len(neighbours)):
        response = neighbours[x][-1]
        if response in classvotes:
            classvotes[response] += 1
        else:
            classvotes[response] = 1
        sortedvotes = sorted(classvotes.items(), key=operator.itemgetter(1), reverse=True)
        return sortedvotes[0][0]

def get_accuracy(testset, predictions):
    correct = 0
    for x in range(len(testset)):
        if testset[x][-1] == predictions[x]:
            correct += 1
    return (correct / float(len(testset))) * 100

def main():
    # 读取数据
    trainset = []
    testset = []
    split = 0.66
    load_dataset('E:/Kaggle/iris.data', 0.66, trainset, testset)
    print ('TrainSet:' + repr(len(trainset)))
    print ('TestSet:' + repr(len(testset)))

    # 作出预测
    predictions = []
    k = 3
    for x in range(len(testset)):
        neighbours = get_neighbours(trainset, testset[x], k)
        result = get_response(neighbours)
        predictions.append(result)
        print (('predicted=' + repr(result) + ', actual=' + repr(testset[x][-1])))
    accuracy = get_accuracy(testset, predictions)
    print ('==================================')
    print (('Accuracy:' + repr(accuracy) + '%'))
    print ('==================================')
main()

TrainSet:96
TestSet:53
predicted='Iris-setosa', actual='Iris-setosa'
predicted='Iris-setosa', actual='Iris-setosa'
predicted='Iris-setosa', actual='Iris-setosa'
predicted='Iris-setosa', actual='Iris-setosa'
predicted='Iris-setosa', actual='Iris-setosa'
predicted='Iris-setosa', actual='Iris-setosa'
predicted='Iris-setosa', actual='Iris-setosa'
predicted='Iris-setosa', actual='Iris-setosa'
predicted='Iris-setosa', actual='Iris-setosa'
predicted='Iris-setosa', actual='Iris-setosa'
predicted='Iris-setosa', actual='Iris-setosa'
predicted='Iris-setosa', actual='Iris-setosa'
predicted='Iris-setosa', actual='Iris-setosa'
predicted='Iris-setosa', actual='Iris-setosa'
predicted='Iris-setosa', actual='Iris-setosa'
predicted='Iris-setosa', actual='Iris-setosa'
predicted='Iris-setosa', actual='Iris-setosa'
predicted='Iris-setosa', actual='Iris-setosa'
predicted='Iris-setosa', actual='Iris-setosa'
predicted='Iris-setosa', actual='Iris-setosa'
predicted='Iris-setosa', actual='Iris-setosa'
predicted='